In [1]:
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Creating a custom prediction routine with Keras

<table align="left">
  <td>
    <a href="https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routine-keras">
      <img src="https://cloud.google.com/_static/images/cloud/icons/favicons/onecloud/super_cloud.png"
           alt="Google Cloud logo" width="32px"> Read on cloud.google.com
    </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/tensorflow/custom-prediction-routine-keras.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/tensorflow/custom-prediction-routine-keras.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

**Beta**

This is a beta release of custom prediction routines. This feature might be changed in backward-incompatible ways and is not subject to any SLA or deprecation policy.

## Overview

This tutorial shows how to deploy a trained Keras model to AI Platform and serve predictions using a [custom prediction routine](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines). This lets you customize how AI Platform responds to each prediction request.

In this example, you will use a custom prediction routine to preprocess
prediction input by scaling it, and to postprocess prediction output by converting [softmax](https://developers.google.com/machine-learning/glossary/#s) probability outputs to label strings.

The tutorial walks through several steps:

* Training a simple Keras model locally (in this notebook)
* Creating and deploy a custom prediction routine to AI Platform
* Serving prediction requests from that deployment

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small dataset that is popular for trying out machine learning techniques. Each instance has four numerical features, which are different measurements of a flower, and a target label that
marks it as one of three types of iris: Iris setosa, Iris versicolour, or Iris virginica.

This tutorial uses [the copy of the Iris dataset included in the
scikit-learn library](https://scikit-learn.org/stable/datasets/index.html#iris-dataset).

### Objective

The goal is to train a model that uses a flower's measurements as input to predict what type of iris it is.

This tutorial focuses more on using this model with AI Platform than on
the design of the model itself.

### Costs

This tutorial uses billable components of Google Cloud Platform (GCP):

* AI Platform
* Cloud Storage

Learn about [AI Platform
pricing](https://cloud.google.com/ml-engine/docs/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

You must do several things before you can train and deploy a model in AI Platform:

* Set up your local development environment.
* Set up a GCP project with billing and the necessary
  APIs enabled.
* Authenticate your GCP account in this notebook.
* Create a Cloud Storage bucket to store your training package and your
  trained model.


### Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already
meets all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

2. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

3. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3.

4. Activate that environment and run `pip install jupyter` in a shell to install
   Jupyter.

5. Run `jupyter notebook` in a shell to launch Jupyter.

6. Open this notebook in the Jupyter Notebook Dashboard.

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project.](https://console.cloud.google.com/cloud-resource-manager)

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the AI Platform ("Cloud Machine Learning Engine") and Compute Engine
APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [2]:
PROJECT_ID = "paradox-mo" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


### Authenticate your GCP account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the GCP Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. From the **Service account** drop-down list, select **New service account**.

3. In the **Service account name** field, enter a name.

4. From the **Role** drop-down list, select
   **Machine Learning Engine > AI Platform Admin** and
   **Storage > Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [0]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS '<path-to-your-service-account-key.json>'

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

To deploy a custom prediction routine, you must upload your trained model
artifacts and your custom code to Cloud Storage.

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets. 

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Cloud
AI Platform services are
available](https://cloud.google.com/ml-engine/docs/tensorflow/regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [3]:
BUCKET_NAME = 'mo_ml' #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Creating gs://mo_ml/...
ServiceException: 409 Bucket mo_ml already exists.


Finally, validate access to your Cloud Storage bucket by examining its contents:


In [5]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://mo_ml/email/
                                 gs://mo_ml/inapp/
                                 gs://mo_ml/lingh/
                                 gs://mo_ml/push/


## Building and training a Keras model

Often, you can't use your data in its raw form to train a machine learning model. Even when you can, preprocessing the data before using it for training can sometimes improve your model.

Assuming that you expect the input for prediction to have the same format as your training data, you must apply identical preprocessing during training and prediction to ensure that your model makes consistent predictions.

In this section, create a preprocessing module and use it as part of training. Then export a preprocessor with characteristics learned during training to use later in your custom prediction routine.

### Install dependencies for local training

Training locally (in the notebook) requires several dependencies:

In [6]:
! pip3 install numpy scikit-learn 'tensorflow>=1.13,<2'

Looking in indexes: https://pypi.org/simple, https://artifactory.spotify.net/artifactory/api/pypi/pypi/simple/
  Using cached https://files.pythonhosted.org/packages/b4/e8/5ececadd9cc220bb783b4ce6ffaa9266925d37ed41237bc23bc530ab4f3d/numpy-1.17.2-cp37-cp37m-macosx_10_6_intel.whl
  Using cached https://files.pythonhosted.org/packages/e9/57/8a9889d49d0d77905af5a7524fb2b468d2ef5fc723684f51f5ca63efed0d/scikit_learn-0.21.3-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ed/11/037887c5cbac5af3124050fb6348e67caa038734cc9673b11c31c8939072/tensorflow-1.14.0-cp37-cp37m-macosx_10_11_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cd/c1/50a758e8247561e58cb87305b1e90b171b8c767b15b12a1734001f41d356/joblib-0.13.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d5/06/1a696649f4b2e706c509cb9333fdc6331fbe71251cede945f9e1fa13ea34/scipy-1.3.1-cp37

### Write your preprocessor

Scaling training data so each numerical feature column has a mean of 0 and a standard deviation of 1 [can improve your model](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data).

Create `preprocess.py`, which contains a class to do this scaling:

In [7]:
%%writefile preprocess.py
import numpy as np

class MySimpleScaler(object):
  def __init__(self):
    self._means = None
    self._stds = None

  def preprocess(self, data):
    if self._means is None: # during training only
      self._means = np.mean(data, axis=0)

    if self._stds is None: # during training only
      self._stds = np.std(data, axis=0)
      if not self._stds.all():
        raise ValueError('At least one column has standard deviation of 0.')

    return (data - self._means) / self._stds

Writing preprocess.py


Notice that an instance of `MySimpleScaler` saves the means and standard deviations of each feature column on first use. Then it uses these summary statistics to scale data it encounters afterward.

This lets you store characteristics of the training distribution and use them for identical preprocessing at prediction time.

### Train your model

Next, use `preprocess.MySimpleScaler` to preprocess the iris data, then train a simple neural network using Keras.

At the end, export your trained Keras model as an HDF5 (`.h5`) file and export
your `MySimpleScaler` instance as a pickle (`.pkl`) file:

In [8]:
import pickle

from sklearn.datasets import load_iris
import tensorflow as tf

from preprocess import MySimpleScaler

iris = load_iris()
scaler = MySimpleScaler()
num_classes = len(iris.target_names)
X = scaler.preprocess(iris.data)
y = tf.keras.utils.to_categorical(iris.target, num_classes=num_classes)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(25, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(25, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
model.compile(
  optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=1)

model.save('model.h5')
with open ('preprocessor.pkl', 'wb') as f:
  pickle.dump(scaler, f)

Epoch 1/10
150/150 [==============================] - 0s 2ms/step - loss: 0.7939 - acc: 0.7000
Epoch 2/10
150/150 [==============================] - 0s 778us/step - loss: 0.4337 - acc: 0.8400
Epoch 3/10
150/150 [==============================] - 0s 766us/step - loss: 0.3221 - acc: 0.8533
Epoch 4/10
150/150 [==============================] - 0s 769us/step - loss: 0.2619 - acc: 0.8800
Epoch 5/10
150/150 [==============================] - 0s 769us/step - loss: 0.2207 - acc: 0.9000
Epoch 6/10
150/150 [==============================] - 0s 766us/step - loss: 0.1745 - acc: 0.9467
Epoch 7/10
150/150 [==============================] - 0s 766us/step - loss: 0.1406 - acc: 0.9667
Epoch 8/10
150/150 [==============================] - 0s 759us/step - loss: 0.1115 - acc: 0.9667
Epoch 9/10
150/150 [==============================] - 0s 762us/step - loss: 0.1040 - acc: 0.9733
Epoch 10/10
150/150 [==============================] - 0s 765us/step - loss: 0.0901 - acc: 0.9733


Note: When deploying a TensorFlow model to AI Platform without a custom prediction routine, you must export the trained model in the `SavedModel` format. When you deploy a custom prediction routine, you are able to export to the HDF5 format instead—or any other format that suits your needs.

## Deploying a custom prediction routine

To deploy a custom prediction routine to serve predictions from your trained model, do the following:

* Create a custom predictor to handle requests
* Package your predictor and your preprocessing module
* Upload your model artifacts and your custom code to Cloud Storage
* Deploy your custom prediction routine to AI Platform

### Create a custom predictor

To deploy a custom prediction routine, you must create a class that implements
the [Predictor interface](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines#predictor-class). This tells AI Platform how to load your model and how to handle prediction requests.

Write the following code to `predictor.py`:

In [9]:
%%writefile predictor.py
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
import tensorflow as tf

class MyPredictor(object):
  def __init__(self, model, preprocessor):
    self._model = model
    self._preprocessor = preprocessor
    self._class_names = load_iris().target_names

  def predict(self, instances, **kwargs):
    inputs = np.asarray(instances)
    preprocessed_inputs = self._preprocessor.preprocess(inputs)
    outputs = self._model.predict(preprocessed_inputs)
    if kwargs.get('probabilities'):
      return outputs.tolist()
    else:
      return [self._class_names[index] for index in np.argmax(outputs, axis=1)]

  @classmethod
  def from_path(cls, model_dir):
    model_path = os.path.join(model_dir, 'model.h5')
    model = tf.keras.models.load_model(model_path)

    preprocessor_path = os.path.join(model_dir, 'preprocessor.pkl')
    with open(preprocessor_path, 'rb') as f:
      preprocessor = pickle.load(f)

    return cls(model, preprocessor)

Writing predictor.py


Notice that, in addition to using the preprocessor that you defined during training, this predictor performs a postprocessing step that converts the neural network's softmax output (an array denoting the probability of each label being the correct one) into the label with the highest probability.

However, if the predictor receives a `probabilities` keyword argument with the value `True`, it
returns the probability array instead. The last part of this tutorial shows how to provide this keyword argument.

### Package your custom code

You must package `predictor.py` and `preprocess.py` as a `.tar.gz` source distribution package and provide the package to AI Platform so it can use your custom code to serve predictions.

Write the following `setup.py` to define your package:

In [10]:
%%writefile setup.py
from setuptools import setup

setup(
    name='my_custom_code',
    version='0.1',
    scripts=['predictor.py', 'preprocess.py'])

Writing setup.py


Then run the following command to create`dist/my_custom_code-0.1.tar.gz`:

In [11]:
! python setup.py sdist --formats=gztar

running sdist
running egg_info
creating my_custom_code.egg-info
writing my_custom_code.egg-info/PKG-INFO
writing top-level names to my_custom_code.egg-info/top_level.txt
writing dependency_links to my_custom_code.egg-info/dependency_links.txt
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'
reading manifest file 'my_custom_code.egg-info/SOURCES.txt'
writing manifest file 'my_custom_code.egg-info/SOURCES.txt'

running check


creating my_custom_code-0.1
creating my_custom_code-0.1/my_custom_code.egg-info
copying files to my_custom_code-0.1...
copying predictor.py -> my_custom_code-0.1
copying preprocess.py -> my_custom_code-0.1
copying setup.py -> my_custom_code-0.1
copying my_custom_code.egg-info/PKG-INFO -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/SOURCES.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/dependency_links.txt -> my_custom_code-0.1/my_custom_code.egg-info
copying my_custom_code.egg-info/top_lev

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, AI Platform needs access to the following files in Cloud Storage:

* `model.h5` (model artifact)
* `preprocessor.pkl` (model artifact)
* `my_custom_code-0.1.tar.gz` (custom code)

Model artifacts must be stored together in a model directory, which your
Predictor can access as [the `model_dir` argument in its `from_path` class
method](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines#predictor-class). The custom
code does not need to be in the same directory. Run the following commands to
upload your files:

In [12]:
! gsutil cp ./dist/my_custom_code-0.1.tar.gz gs://$BUCKET_NAME/lingh/custom_prediction_routine_tutorial/my_custom_code-0.1.tar.gz
! gsutil cp model.h5 preprocessor.pkl gs://$BUCKET_NAME/lingh/custom_prediction_routine_tutorial/model/

Copying file://./dist/my_custom_code-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  1.3 KiB/  1.3 KiB]                                                
Operation completed over 1 objects/1.3 KiB.                                      
Copying file://model.h5 [Content-Type=application/octet-stream]...
Copying file://preprocessor.pkl [Content-Type=application/octet-stream]...      
- [2 files][ 37.8 KiB/ 37.8 KiB]                                                
Operation completed over 2 objects/37.8 KiB.                                     


### Deploy your custom prediction routine

Create [a *model* resource and a *version* resource](https://cloud.google.com/ml-engine/docs/tensorflow/projects-models-versions-jobs) to deploy your custom prediction routine. First define variables with your resource names:

In [13]:
MODEL_NAME = 'IrisPredictor'
VERSION_NAME = 'v1'

Then create your model:

In [14]:
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

Created ml engine model [projects/paradox-mo/models/IrisPredictor].


Next, create a version. In this step, provide paths to the artifacts and custom code you uploaded to Cloud Storage:

In [15]:
# --quiet automatically installs the beta component if it isn't already installed 
! gcloud --quiet beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://$BUCKET_NAME/lingh/custom_prediction_routine_tutorial/model/ \
  --package-uris gs://$BUCKET_NAME/lingh/custom_prediction_routine_tutorial/my_custom_code-0.1.tar.gz \
  --prediction-class predictor.MyPredictor

Creating version (this might take a few minutes)......failed.                  
ERROR: (gcloud.beta.ai-platform.versions.create) Create Version failed. Bad model detected with error:  "Failed to load model: Unexpected error when loading the model: You are trying to load a weight file containing 3 layers into a model with 0 layers. (Error code: 0)"


Learn more about [the options you must specify](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines#deploy_your_custom_prediction_routine) when you deploy a custom prediction routine.

## Serving online predictions

Try out your deployment by sending an online prediction request. First, install the Google APIs Client Library for Python:

In [0]:
! pip install --upgrade google-api-python-client

Then send two instances of iris data to your deployed version:

In [0]:
import googleapiclient.discovery

instances = [
  [6.7, 3.1, 4.7, 1.5],
  [4.6, 3.1, 1.5, 0.2],
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

Note: This code uses the credentials you set up during the authentication step to make the online prediction request.

### Sending keyword arguments

When you send a prediction request to a custom prediction routine, you can provide [additional fields on your request body](https://cloud.google.com/ml-engine/docs/v1/predict-request). The Predictor's `predict` method receives these as fields of the `**kwargs` dictionary.

The following code sends the same request as before, but this time it adds a `probabilities` field to the request body:

In [0]:
response = service.projects().predict(
    name=name,
    body={'instances': instances, 'probabilities': True}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

## Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by running the following commands:

In [0]:
# Delete version resource
! gcloud ai-platform versions delete $VERSION_NAME --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$BUCKET_NAME/custom_prediction_routine_tutorial

## What's next

* Read more about [custom prediction routines](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines).
* Learn about how to [export a model](https://cloud.google.com/ml-engine/docs/tensorflow/exporting-for-prediction) to serve predictions without a custom prediction routine.